[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramirezlab/Farmacoinformatica/blob/main/02_Sesiones-practicas/07_Sesion-VII/Smina.ipynb)

# Basic Protein-ligand Docking with Smina (Re-Docking)
Smina protein-ligand re-docking   
* Laboratorio de diseño de fármacos Nanocell
* https://www.nanocell.cl
* Mr. Ignacio Martínez Valenzuela
* v.1
* November 2021

# Smina
https://sourceforge.net/projects/smina/
https://github.com/mwojcikowski/smina
## Requirements
* test_version = smina 2020.12.10 conda-forge:b08c07c   Built Jan 30 2021.  Based on AutoDock Vina 1.1.2.
* conda install -c conda-forge smina
* conda install -c conda-forge rdkit
* conda install -c conda-forge py3dmol
* conda install -c conda-forge spyrmsd
* conda install -c conda-forge prody
* or
* Recomended $\to$ conda create -n smina -c conda-forge smina rdkit py3dmol spyrmsd prody

#Requeriments Installation

In [ ]:
!pip install rdkit
!pip install spyrmsd
!pip install prody
!pip install py3Dmol # 3D Molecular Visualizer
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -q -y -c openbabel openbabel

# Download Smina

In [ ]:
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static

In [ ]:
 #Permissions
!chmod u+x smina.static

In [ ]:
#Usage and commands
!./smina.static

## Select Protein Receptor

In [ ]:
# We're going to use crystalographic structure of Tau-Tubulin Kinase 1 as receptor
#If you want to use another structure you need to change the pdb_code
pdb_code = '4BTK'

In [ ]:
#Download the structure from rscb, pdb database
! wget https://files.rcsb.org/download/4BTK.pdb
#Extract chain A
! grep -i ' A ' 4BTK.pdb > 4BTK_A.pdb

## Extract Ligand from PDB

* In this case we´re going to use the same ligand from the structure of receptor to perform docking
* This is called Re-docking

In [ ]:
#Extract ligand from PDB
!grep -i 'DTQ' 4BTK.pdb > 4BTK_ligand.pdb

## Clean Receptor
* We need clean the pdb of all heteroatoms (ligands, waters, ions, precipitant agent) to use it in the docking program

In [ ]:
#Clean the strcture of heteroatoms
!grep -v HETATM 4BTK_A.pdb > 4BTK_clean.pdb

## Convert to PDBQT format
* Format which includes atomic partial charges and atom types to the coordinates pdb file

In [ ]:
!obabel 4BTK_clean.pdb -xr -O 4BTK_clean.pdbqt
!obabel 4BTK_ligand.pdb -O 4BTK_ligand.pdbqt

## Smina Rigid Receptor Docking

In [ ]:
#Smina Rigid Docking
!./smina.static --cpu 12 --seed 0 --exhaustiveness 32 --autobox_add +6 --autobox_ligand 4BTK_ligand.pdbqt -r 4BTK_clean.pdbqt -l 4BTK_ligand.pdbqt -o 4BTK_docking.sdf --log Rigid_results.log

## Visualization

* Best result or pose (cyan) vs crystallographic ligand (magenta)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import py3Dmol

#Parameters

view = py3Dmol.view(width=1000, height=600)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

# Receptor

view.addModel(open('{0}_clean.pdb'.format(pdb_code),'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

# Reference Ligand

view.addModel(open('{0}_ligand.pdb'.format(pdb_code),'r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

# Docking Result

results=Chem.SDMolSupplier('{0}_docking.sdf'.format(pdb_code))

p=Chem.MolToMolBlock(results[0],False)  # [0] give you the first result from docking, to view another change this value
# Print Score

print('Reference: Magenta | Smina Pose: Cyan')
print ('Score: {}'.format(results[0].GetProp('minimizedAffinity')))  # If change docking result above, change this value too

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})


#Visualization
view.zoomTo()
view.animate({'loop': "forward"})
view.rotate(-375,'y',1);
view.rotate(250,'x',1);
view.rotate(150,'z',1);
view.center({'resn':'DTQ'})
view.zoom(5,1)
view.render()

## RMSD Measurement

In [ ]:
from spyrmsd import io, rmsd

#Load Reference

ref = io.loadmol('{0}_ligand.pdb'.format(pdb_code))

coords_ref = ref.coordinates
anum_ref = ref.atomicnums
adj_ref = ref.adjacency_matrix

#Load Smina results

mols = io.loadallmols('{0}_docking.sdf'.format(pdb_code))

for mol in mols:      #Remove the hydrogens from the poses to compare them with the reference which has no hydrogens
    mol.strip()

coords = [mol.coordinates for mol in mols]
anum = mols[0].atomicnums
adj = mols[0].adjacency_matrix

In [ ]:
RMSD = rmsd.symmrmsd(coords_ref,coords,anum_ref,anum,adj_ref,adj)
print(RMSD)

## Flexible Receptor Docking with Smina
* --flexdist_ligand $\to$  Ligand reference to set flexible residues
* --flexdist $\to$ Distance from flexdist_ligand at which residues are considered as flexibles
* With --flexres chain:residue $\to$ you can select manually wich residues want to set as flexible
* out_flex $\to$ file with flexible residues of docking results
* Recomendations:
     * $\to$ increase exhaustiveness
     * $\to$ scan flexres with flexdist_ligand and select residues of interest
     * $\to$ increase autobox_add (not very much)

In [ ]:
#Smina Flex Docking
!./smina.static --cpu 12 --seed 0 --exhaustiveness 64 --flexdist_ligand 4BTK_ligand.pdbqt --flexdist 3 --autobox_add +7 --autobox_ligand 4BTK_ligand.pdbqt -r 4BTK_clean.pdbqt -l 4BTK_ligand.pdbqt -o 4BTK_flex_docking.sdf --out_flex 4BTK_flex_residues.pdb --log Flex_results.log

In [ ]:
!wget https://raw.githubusercontent.com/ramirezlab/WIKI/master/Docking_and_Virtual_Screening/Docking-SMINA/smina/run/makeflex.py

### Merge Flex Residues from docking to origen structure
*Download makeflex.py with wget
* Download makflex.py from https://github.com/gnina/gnina/blob/master/scripts/
* Arguments:

             rigidname = args.rigid

             flexname  = args.flex
             
             outfile   = args.out

In [ ]:
!python makeflex.py 4BTK_clean.pdb 4BTK_flex_residues.pdb 4BTK_merge.pdb

In [ ]:
import py3Dmol

#Parameters

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

#Protein with merged residues

view.addModel(open('{0}_merge.pdb'.format(pdb_code),'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

#Reference Ligand

view.addModel(open('{0}_ligand.pdb'.format(pdb_code),'r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

# Docking Results

results=Chem.SDMolSupplier('{0}_flex_docking.sdf'.format(pdb_code))

p=Chem.MolToMolBlock(results[1],False)

print('Reference: Magenta | Smina Pose: Cyan | Smina Flexres: Yellow ')
print ('Score: {}'.format(results[0].GetProp('minimizedAffinity')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

#Flexible Residues

view.addModel(open('{0}_flex_residues.pdb'.format(pdb_code),'r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'yellowCarbon','radius':0.2}})

#Visualization
view.zoomTo()
view.animate({'loop': "forward"})
view.rotate(-375,'y',1);
view.rotate(250,'x',1);
view.rotate(150,'z',1);
view.center({'resn':'DTQ'})
view.zoom(4,1)
view.render()

# RMSD measurement

In [ ]:
from spyrmsd import io, rmsd

view.zoomTo()
view.rotate(480,'y',1);
view.rotate(-270,'x',1);
view.center({'resn':'40D'})
view.zoom(6,1000)
view.render()
#Load Reference

ref = io.loadmol('{0}_ligand.pdb'.format(pdb_code))

coords_ref = ref.coordinates
anum_ref = ref.atomicnums
adj_ref = ref.adjacency_matrix

#Load Smina results

mols = io.loadallmols('{0}_flex_docking.sdf'.format(pdb_code))

for mol in mols:      #Remove the hydrogens from the poses to compare them with the reference which has no hydrogens
    mol.strip()

coords = [mol.coordinates for mol in mols]
anum = mols[0].atomicnums
adj = mols[0].adjacency_matrix

In [ ]:
RMSD = rmsd.symmrmsd(coords_ref,coords,anum_ref,anum,adj_ref,adj)
print(RMSD)